# **Penting**

*   Pada Notebook ini, Anda hanya perlu mengerjakan code pada bagian **logic.py** saja. Anda tidak diwajibkan untuk mengubah atau menambahkan **app.py** yang digunakan untuk membangun interface Streamlit.
*   Namun, jika Anda memiliki preferensi lain atau ingin mengubah struktur code pada logic ataupun pada interface Streamlit, itu **DIPERSILAHKAN** saja, tetapi pastikan untuk memenuhi kriteria yang telah ditetapkan pada intruksi submission
*   Jika Anda tidak ingin mengubah apapun dan ingin mengikuti template, tugas Anda hanyalah melengkapi code yang rumpang pada bagian yang sudah ditandai "________" saja.



# **Prepare Dependencies**

In [42]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q torch
!pip install -q diffusers
!pip install -q transformers
!pip install -q streamlit_drawable_canvas==0.8.0

In [43]:
from pyngrok import ngrok
# import subprocess

# **Streamlit**

## logic.py (**Basic**)

In [44]:
%%writefile logic.py
import torch
import gc
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionInpaintPipeline,
    StableDiffusionImg2ImgPipeline,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler,
    DDIMScheduler
)
from PIL import Image, ImageFilter, ImageDraw
import numpy as np

# MODEL LOADER (OPTIMIZED FOR M1 MAC & DICODING CONVENTIONS)
def load_models_cached():
    device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
    dtype = torch.float16 if device == "cuda" else torch.float32
    print(f"Loading models to {device} using {dtype}")

    pipe_txt2img = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", 
        torch_dtype=dtype, 
        use_safetensors=False,
        safety_checker=None,
        requires_safety_checker=False
    ).to(device)
    pipe_txt2img.enable_attention_slicing()

    pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting", 
        torch_dtype=dtype, 
        use_safetensors=False,
        safety_checker=None,
        requires_safety_checker=False
    ).to(device)
    pipe_inpaint.enable_attention_slicing()

    return pipe_txt2img, pipe_inpaint

def flush_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def set_scheduler(pipe, name):
    if name == "Euler A":
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif name == "DPM++":
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    elif name == "DDIM":
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    return pipe

def generate_image(pipe, prompt, neg_prompt, seed, steps, cfg, num_images=1, scheduler_name="Euler A"):
    device = pipe.device
    pipe = set_scheduler(pipe, scheduler_name)
    generator = torch.Generator(device=device).manual_seed(seed)

    # --- STAGE 1: BASE GENERATION ---
    base_output = pipe(
        prompt=prompt,
        negative_prompt=neg_prompt,
        num_inference_steps=steps,
        guidance_scale=cfg,
        num_images_per_prompt=num_images,
        generator=generator
    )
    base_images = base_output.images

    # --- STAGE 2: REFINER (Refinement Logic) ---
    pipe_refiner = StableDiffusionImg2ImgPipeline(
        vae=pipe.vae,
        text_encoder=pipe.text_encoder,
        tokenizer=pipe.tokenizer,
        unet=pipe.unet,
        scheduler=pipe.scheduler,
        feature_extractor=pipe.feature_extractor,
        safety_checker=None
    ).to(device)

    result = pipe_refiner(
        prompt=[prompt] * len(base_images),
        negative_prompt=[neg_prompt] * len(base_images),
        image=base_images,
        num_inference_steps=steps,
        guidance_scale=cfg,
        generator=generator,
        strength=0.8
    ).images

    return result

def inpaint_engine(pipe, image, mask, prompt, negative_prompt="", seed=9, is_outpainting=False):
    device = pipe.device
    strength = 1.0 if is_outpainting else 0.9
    steps = 50 if is_outpainting else 30

    if image.mode != "RGB": image = image.convert("RGB")
    if mask.mode != "L": mask = mask.convert("L")
    image = image.resize((512, 512))
    mask = mask.resize((512, 512), resample=Image.NEAREST)

    generator = torch.Generator(device=device).manual_seed(seed)
    result = pipe(
        prompt=prompt, 
        negative_prompt=negative_prompt,
        image=image, 
        mask_image=mask, 
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=7.5,
        generator=generator
    ).images[0]
    return result

def prepare_outpainting(image, direction="right", expand_pixels=256):
    init_image = image.convert("RGB").resize((512, 512))
    w, h = init_image.size
    
    if direction == "zoom_out":
        padding_scale = (w + expand_pixels) / w
        new_width = int(w * padding_scale)
        new_height = int(h * padding_scale)
        canvas = Image.new("RGB", (new_width, new_height), (0, 0, 0))
        x = (new_width - w) // 2
        y = (new_height - h) // 2
        canvas.paste(init_image, (x, y))
        mask = Image.new("L", (new_width, new_height), 255)
        mask_content = Image.new("L", (w, h), 0)
        mask.paste(mask_content, (x, y))
    else:
        if direction == "left":
            new_size = (w + expand_pixels, h)
            paste_pos = (expand_pixels, 0)
        elif direction == "right":
            new_size = (w + expand_pixels, h)
            paste_pos = (0, 0)
        elif direction == "up":
            new_size = (w, h + expand_pixels)
            paste_pos = (0, expand_pixels)
        elif direction == "down":
            new_size = (w, h + expand_pixels)
            paste_pos = (0, 0)
        else:
            new_size = (w + expand_pixels, h)
            paste_pos = (0, 0)

        canvas = Image.new("RGB", new_size, (0, 0, 0))
        canvas.paste(init_image, paste_pos)
        mask = Image.new("L", new_size, 255)
        mask_content = Image.new("L", (w, h), 0)
        mask.paste(mask_content, paste_pos)
    
    return canvas.resize((512, 512)), mask.resize((512, 512))


Overwriting logic.py


In [45]:
# Appending logic (Handled in main logic cell)

Appending to logic.py


## logic.py (**Skilled**)

In [46]:
# Appending logic (Handled in main logic cell)

Appending to logic.py


## logic.py (**Advanced**)

In [47]:
# Appending logic (Handled in main logic cell)

Appending to logic.py


## app.py
Anda **TIDAK perlu mengubah atau menambahkan** apapun pada file **app.py** ini, cukup **jalankan** saja.

In [48]:
%%writefile app.py
import streamlit as st
import torch
import numpy as np
from PIL import Image
from streamlit_drawable_canvas import st_canvas
import logic
from PIL import Image, ImageFilter

st.set_page_config(page_title="StudioAI", layout="wide", page_icon="🎨")

@st.cache_resource
def get_models():
    return logic.load_models_cached()

try:
    pipe_txt2img, pipe_inpaint = get_models()
except Exception as e:
    st.error(f"Error loading models: {e}")
    st.stop()

st.title("🎨 StudioAI: Image Generation Suite")

with st.sidebar:
    st.header("⚙️ Parameters")
    steps = st.slider("Quality Steps", 15, 50, 30)
    cfg = st.slider("Creativity (CFG)", 1.0, 20.0, 7.5)
    seed = st.number_input("Seed Control", value=222)
    scheduler_name = st.selectbox("Scheduler", ["Euler A", "DPM++", "DDIM"])
    num_images = st.slider("Batch Size", 1, 4, 4)

    if st.button("🧹 Flush RAM"):
        logic.flush_memory()
        st.toast("Memory Cleared!")

tab_gen, tab_edit = st.tabs(["✨ GENERATE", "🛠️ EDIT"])

with tab_gen:
    c1, c2 = st.columns([1, 1], gap="large")
    with c1:
        st.subheader("Input Blueprint")
        with st.form(key="gen_form"):
            prompt = st.text_area("Prompt", "an astronaut on the moon with planet earth visible in the background, highly detailed, 8k resolution", height=150)
            neg_prompt = st.text_input("Negative Prompt", "photorealistic, realistic, photograph, 3d render, messy, blurry, low quality, bad art, ugly, sketch, grainy, unfinished, chromatic aberration")
            submit_gen = st.form_submit_button("🚀 Initialize Generation", type="primary")

        if submit_gen:
            with st.spinner("Processing..."):
                logic.flush_memory()
                generated_list = logic.generate_image(pipe_txt2img, prompt, neg_prompt, seed, steps, cfg, num_images, scheduler_name)
                st.session_state['generated_images'] = generated_list
                if generated_list: st.session_state['current_image'] = generated_list[0]
            st.rerun()

    with c2:
        st.subheader("Visual Output (2x2 Grid)")
        if 'generated_images' in st.session_state:
            imgs = st.session_state['generated_images']
            if len(imgs) > 1:
                cols = st.columns(2)
                for idx, img in enumerate(imgs):
                    with cols[idx % 2]:
                        st.image(img, caption=f"Variant #{idx+1}", use_container_width=True)
                        if st.button(f"Edit #{idx+1}", key=f"sel_{idx}"):
                             st.session_state['current_image'] = img
                             st.toast(f"✅ Image #{idx+1} Selected!")
            elif len(imgs) == 1:
                st.image(imgs[0], caption="Single Result", use_container_width=True)
        else:
            st.info("👈 Enter your prompt on the left.")

with tab_edit:
    if 'current_image' in st.session_state:
        source_img = st.session_state['current_image'].resize((512, 512))
        mode = st.radio("Tool Select:", ["Inpaint", "Outpaint"], horizontal=True)
        if mode == "Inpaint":
            col_tools, col_result = st.columns([1, 1])
            if 'canvas_key' not in st.session_state: st.session_state['canvas_key'] = 0
            with col_tools:
                st.subheader("✍️ Draw Mask")
                canvas_result = st_canvas(fill_color="rgba(255, 255, 255, 1.0)", stroke_width=20, stroke_color="#FFFFFF",
                    background_image=source_img, height=512, width=512, drawing_mode="freedraw", key=f"canvas_{st.session_state['canvas_key']}")
            with col_result:
                st.subheader("Inpaint Parameters")
                with st.form("inpaint_input"):
                    edit_prompt = st.text_input("Modify with...", "a detailed broken satellite floating in space, high resolution, cinematic, sharp focus")
                    inpaint_seed = st.number_input("Seed", value=9)
                    submit_inpaint = st.form_submit_button("⚡ Execute", type="primary")
                if submit_inpaint:
                    if canvas_result.image_data is not None and np.max(canvas_result.image_data) > 0:
                        with st.spinner("Inpainting..."):
                            mask_data = canvas_result.image_data[:, :, 3]
                            mask_data[mask_data > 0] = 255
                            mask_image = Image.fromarray(mask_data.astype('uint8'), mode='L')
                            res_img = logic.inpaint_engine(pipe_inpaint, source_img, mask_image, edit_prompt, seed=inpaint_seed)
                            st.session_state['current_image'] = res_img
                            st.session_state['canvas_key'] += 1
                            st.rerun()
        else:
            c_out_1, c_out_2 = st.columns([1, 1])
            with c_out_1: st.image(source_img, use_container_width=True)
            with c_out_2:
                with st.form("outpaint_input"):
                    direction = st.selectbox("Expansion", ["zoom_out", "right", "left", "up", "down"])
                    out_prompt = st.text_input("Background prompt...", "vast cosmic landscape, stars")
                    submit_outpaint = st.form_submit_button("🔍 Expand", type="primary")
                if submit_outpaint:
                    with st.spinner("Expanding..."):
                        c_ready, m_ready = logic.prepare_outpainting(source_img, direction=direction)
                        res = logic.inpaint_engine(pipe_inpaint, c_ready, m_ready, out_prompt, seed=9, is_outpainting=True)
                        st.session_state['current_image'] = res
                        st.rerun()
    else: st.info("👈 Generate an image first.")


Overwriting app.py


# **Menggunakan *Ngrok* Untuk Deployment**

## **Konfigurasi Autentikasi Ngrok dan Menjalankan Aplikasi Streamlit**
Cell ini digunakan untuk mengatur *authentication token Ngrok* dan menjalankan aplikasi Streamlit secara lokal. Token diperlukan agar *Ngrok* dapat membuat tunnel dengan akun pengguna. Setelah token diatur, aplikasi Streamlit dijalankan menggunakan subprocess sehingga server lokal aktif di background.

Apabila Anda belum mengetahui cara mendapatkan *authentication token Ngrok* milik Anda sendiri, silahkan baca pada bagian **tips and tricks submission**.

In [49]:
from google.colab import userdata
from pyngrok import ngrok
import subprocess

# Mengambil token yang kamu simpan dengan nama 'NG_TOKEN' di Secrets Colab
auth_token = userdata.get('NG_TOKEN')

# Konfigurasi Ngrok
ngrok.set_auth_token(auth_token)

# Menjalankan Streamlit di background
# Pastikan file app.py dan logic.py sudah dibuat sebelumnya
subprocess.Popen(["streamlit", "run", "app.py"])
print("Server Streamlit sedang dijalankan di background...")

Server Streamlit sedang dijalankan di background...


## **Membuat Public URL**
Cell ini membuat *tunnel Ngrok* ke port lokal tempat Streamlit berjalan (default: 8501). *Ngrok* kemudian menghasilkan public URL yang bisa diakses dari internet, sehingga aplikasi Streamlit dapat dibuka tanpa harus berada di jaringan lokal yang sama.

In [50]:
!ls -l app.py logic.py

public_url = ngrok.connect(8501).public_url
print(public_url)

-rw-r--r-- 1 root root 9371 Feb 27 04:15 app.py
-rw-r--r-- 1 root root 5401 Feb 27 04:15 logic.py
https://maddeningly-subfastigiated-maryland.ngrok-free.dev


Apabila Anda mengalami limit endpoint usage pada Ngrok, jalankan hidden cell di bawah ini!

## **Menutup Semua Tunnel Ngrok yang Aktif**
Cell ini digunakan untuk menghentikan seluruh koneksi Ngrok yang masih aktif.

In [51]:
#ngrok.kill()
#print("All active ngrok tunnels have been closed.")